<a href="https://colab.research.google.com/github/PetchMa/deeplearning_fundamentals/blob/main/Neural_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build Neuarl Ordinary Differential Equations

Neural ODE's are a new approach to simple multi layered perceptron by feeding it dynamics of a system instead. To begin we need to start off with a simple multi layered perceptron again... 

Here are some good notes documenting this approach
[link](http://implicit-layers-tutorial.org/neural_odes/)


In [3]:
import jax.numpy as jnp
from jax.nn import relu
from jax import jit, grad

def mlp(params, inputs):
  # A multi-layer perceptron, i.e. a fully-connected neural network.
  for w, b in params:
    outputs = jnp.dot(inputs, w) + b  # Linear transform
    inputs = relu(outputs)        # Nonlinearity
  return outputs

# Dynamics 
Now we will try to understand the dynamics 

In [4]:
def nn_dynamics(state, time, params):
  state_and_time = jnp.hstack([state, jnp.array(time)])
  return mlp(params, state_and_time)

In [5]:
from jax.experimental.ode import odeint

def odenet(params, input):
  start_and_end_times = jnp.array([0.0, 1.0])
  init_state, final_state = odeint(nn_dynamics, input, start_and_end_times, params)
  return final_state

In [6]:
from jax import vmap
batched_odenet = vmap(odenet, in_axes=(None, 0))